<a href="https://colab.research.google.com/github/Sandeepkumaramgothu/ASE_practice/blob/main/Spect_2_Gun_Other.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install --upgrade keras
!pip install --upgrade keras tensorflow
import os
import math
import numpy as np
import librosa
import json
import pandas as pd
from termcolor import colored
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
!pip install tensorboard
!pip install visualkeras
import visualkeras
from keras.utils import plot_model
from datetime import datetime
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn import svm
from sklearn.svm import SVC
import pickle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from IPython.display import Audio
from scipy.io import wavfile
import scipy
import soundfile as sf
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report
import random



In [ ]:
import os
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt

def create_mel_spectrogram(audio_file, output_dir, sr=16000, n_fft=2048, hop_length=512, n_mels=128):
    y, sr = librosa.load(audio_file, sr=sr)
    mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
    mel_spec_db = librosa.amplitude_to_db(mel_spec, ref=np.max)

    plt.figure(figsize=(10, 4))
    librosa.display.specshow(mel_spec_db, sr=sr, hop_length=hop_length, x_axis='time', y_axis='mel')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Mel Spectrogram')
    plt.tight_layout()

    filename = os.path.basename(audio_file).split('.')[0] + '_mel.png'
    save_path = os.path.join(output_dir, filename)
    plt.savefig(save_path)
    plt.close()

    print(f'Mel spectrogram saved as {filename}')

def process_directory(audio_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    for file in os.listdir(audio_dir):
        if file.endswith('.wav') or file.endswith('.mp3'):
            audio_file = os.path.join(audio_dir, file)
            create_mel_spectrogram(audio_file, output_dir)

if __name__ == '__main__':
    # Define directories for training and testing audio and mel_images
    base_dir = '/content/drive/MyDrive/DataSet/Dataset_Version_2'
    categories = ['Train', 'Test']

    for category in categories:
        audio_dir = os.path.join(base_dir, category, 'Sound')
        output_dir = os.path.join(base_dir, category, 'Mel_Images')

        if category == 'Train':
            # Process only Gun_Shots for training
            process_directory(os.path.join(audio_dir, 'Gun_Shots'), os.path.join(output_dir, 'Gun_Shots'))
        elif category == 'Test':
            # Process Gun_Shots and Other_Sounds for testing
            process_directory(os.path.join(audio_dir, 'Gun_Shots'), os.path.join(output_dir, 'Gun_Shots'))
            process_directory(os.path.join(audio_dir, 'Other_Sounds'), os.path.join(output_dir, 'Other_Sounds'))


Mel spectrogram saved as 3 (12)_mel.png
Mel spectrogram saved as 3 (16)_mel.png
Mel spectrogram saved as 3 (22)_mel.png
Mel spectrogram saved as 3 (25)_mel.png
Mel spectrogram saved as 3 (11)_mel.png
Mel spectrogram saved as 3 (2)_mel.png
Mel spectrogram saved as 3 (24)_mel.png
Mel spectrogram saved as 3 (10)_mel.png
Mel spectrogram saved as 3 (15)_mel.png
Mel spectrogram saved as 3 (17)_mel.png
Mel spectrogram saved as 3 (23)_mel.png
Mel spectrogram saved as 3 (18)_mel.png
Mel spectrogram saved as 3 (14)_mel.png
Mel spectrogram saved as 3 (26)_mel.png
Mel spectrogram saved as 3 (13)_mel.png
Mel spectrogram saved as 3 (20)_mel.png
Mel spectrogram saved as 3 (19)_mel.png
Mel spectrogram saved as 3 (21)_mel.png
Mel spectrogram saved as 3 (1)_mel.png
Mel spectrogram saved as 3 (29)_mel.png
Mel spectrogram saved as 3 (27)_mel.png
Mel spectrogram saved as 3 (28)_mel.png
Mel spectrogram saved as 3 (30)_mel.png
Mel spectrogram saved as 3 (3)_mel.png
Mel spectrogram saved as 3 (4)_mel.png
Mel 

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

# Assuming base directory is defined as below from previous code
base_dir = '/content/drive/MyDrive/DataSet/Dataset_Version_2/'

# Updated directories for the new folder structure
train_dir = f'{base_dir}Train/Mel_Images/'
test_dir = f'{base_dir}Test/Mel_Images/'


height = 128
n_fft = 2048
hop_length = 512
sr = 16000
T = 1

# Calculating the width from the hop_length and other parameters
hop_size_sec = hop_length / sr
audio_length_samples = T * sr
width = 1 + int((audio_length_samples - n_fft) / hop_length)
channels = 3  # Assuming mel spectrograms are stored as RGB images

# Setting up the data generators with data augmentation for the training set
train_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# No data augmentation for the test set
test_datagen = ImageDataGenerator()

batch_size = 32

# Creating generators that will feed the CNN
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='binary')

# Define the CNN architecture
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(height, width, channels)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Changed for binary classification
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Changed for binary classification
              metrics=['accuracy'])

# Training the model
epochs = 20
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=1,
        verbose=1)

# Evaluating the model on the test data
loss, accuracy = model.evaluate(test_generator)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Saving the trained model
model.save(f'{base_dir}el-amadi_2000_cnn_mcc_standardized.keras')


Found 90 images belonging to 1 classes.
Found 48 images belonging to 2 classes.
Epoch 1/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 0.6104 - loss: 8.5283
Epoch 2/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 563ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 3/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 558ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 4/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 622ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 5/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 625ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 6/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 570ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 7/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 616ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 8/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 761ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 9/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 529ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 10/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 532ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 11/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 536ms/

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Define paths and model parameters
test_directory = '/content/drive/MyDrive/DataSet/Dataset_Version_2/Test/Mel_Images/'
model_path = '/content/drive/MyDrive/DataSet/Dataset_Version_2/el-amadi_2000_cnn_mcc_standardized.keras'

# Load the pre-trained CNN model
model = tf.keras.models.load_model(model_path)

# Audio processing parameters
sample_rate = 16000
fft_points = 2048
hop_length = 512
audio_duration_seconds = 1

# Calculate spectrogram width
audio_length_samples = sample_rate * audio_duration_seconds
spectrogram_width = 1 + int((audio_length_samples - fft_points) / hop_length)

# Image and training parameters
image_height = 128
image_width = spectrogram_width
num_channels = 3
batch_size = 32

# Setup test data generator
test_data_generator = ImageDataGenerator()

# Create test generator
test_generator = test_data_generator.flow_from_directory(
    test_directory,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='binary',  # Assuming binary classification
    shuffle=False)

# Evaluate the model
evaluation_results = model.evaluate(test_generator)
print(f"Test Loss: {evaluation_results[0]}")
print(f"Test Accuracy: {evaluation_results[1]}")

# Check for additional metrics
if len(evaluation_results) > 2:
    print(f"Test Precision: {evaluation_results[2]}")
    print(f"Test Recall: {evaluation_results[3]}")
    print(f"Test F1 Score: {evaluation_results[4]}")

# Predict on the test data
predictions = model.predict(test_generator)
predicted_class_indices = np.argmax(predictions, axis=1)

# Retrieve true class labels from the test generator
true_class_indices = test_generator.classes

# Compute confusion matrix
confusion_mat = confusion_matrix(true_class_indices, predicted_class_indices)
print("\nConfusion Matrix:")
print(confusion_mat)

# Retrieve class labels for clarity in reporting
class_labels = list(test_generator.class_indices.keys())

# Calculate and display accuracy for each class
class_accuracy = {}
for class_index, class_name in enumerate(class_labels):
    correct_class_indices = np.where(true_class_indices == class_index)[0]
    correct_predictions = np.sum(predicted_class_indices[correct_class_indices] == class_index)
    total_predictions = len(correct_class_indices)
    class_accuracy[class_name] = (correct_predictions / total_predictions) if total_predictions > 0 else 0.0

print("\nAccuracy for each class:")
for class_name, accuracy in class_accuracy.items():
    print(f"{class_name}: {accuracy:.4f}")

# Print a detailed classification report
print("\nClassification Report:")
print(classification_report(true_class_indices, predicted_class_indices, target_names=class_labels))


Found 48 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 202ms/step - accuracy: 0.2083 - loss: 1741.7455
Test Loss: 1514.2945556640625
Test Accuracy: 0.3125
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 224ms/step

Confusion Matrix:
[[33  0]
 [15  0]]

Accuracy for each class:
Gun_Shots: 1.0000
Other_Sounds: 0.0000

Classification Report:
              precision    recall  f1-score   support

   Gun_Shots       0.69      1.00      0.81        33
Other_Sounds       0.00      0.00      0.00        15

    accuracy                           0.69        48
   macro avg       0.34      0.50      0.41        48
weighted avg       0.47      0.69      0.56        48



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
